# 1.Project Title: [Data Classificcation]
___

#### a. Introduction

- **Objective:** Clearly state the goal of your project. What problem are you trying to solve?
- **Background:** Provide context on why this problem is important or interesting. Mention any relevant research, datasets, or industry relevance.
- **Scope:** Define the boundaries of your project. What will be included, and what will be out of scope?

#### b. Project Overview

- **Project Summary:** A brief overview of the project, including the main steps you will take to achieve the objective.
- **Milestones:** Outline the key milestones or phases of the project. For example:
  - Data Collection
  - Data Preprocessing
  - Model Selection
  - Model Training and Evaluation
  - Results and Conclusion


#### c. About the Author

- **Name:** [Ahmed Ferganey]
- **Background:** Junior Data Scientist and Machine Learning Engineer with a strong foundation in embedded systems, industrial engineering, and supply chain management. Knowledgeable in statistical analysis, NLP, Computer Vision, and deep learning, with hands-on experience in Python, SQL, and Docker.
- **Motivation:** Why are you interested in this project? What do you hope to learn or achieve?
- **Contact:** [LinkedIn acc](https://www.linkedin.com/in/ahmed-ferganey/)



#### d. Tools and Technologies

- **Programming Languages:** List the programming languages you will use (e.g., Python).
- **Libraries and Frameworks:** List the specific libraries and frameworks you will use (e.g., TensorFlow, scikit-learn).
- **Software and Tools:** Mention any software or tools necessary for the project (e.g., Jupyter Notebook, Git).

#### e. Dataset Description

- **Dataset Name:** [Name of the Dataset]
- **Source:** Where did you obtain the dataset? Include a link if possible.
- **Description:** Briefly describe the dataset, including the number of features, the target variable, and any other important details.
- **Data Preprocessing:** Outline any preprocessing steps you anticipate, such as data cleaning, normalization, or feature engineering.

#### f. Methodology

- **Model Selection:** Describe the types of models you are considering and why.
- **Evaluation Metrics:** Define how you will evaluate your models' performance (e.g., accuracy, F1-score).
- **Validation Strategy:** Explain how you will validate your models, such as cross-validation or a 


### 2. importing libraries
___



In [1]:
import os 
import io
import sys
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile , f_classif ,SelectKBest
from sklearn.feature_selection import chi2 , f_classif 
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from tqdm import tqdm

### 3. reading the raw data
___

In [2]:
# Path to the main directory
path_ = r'/media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new'

# Paths for saving files
merged_output_path = r'/media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/merged_output.csv'
headered_output_path = r'/media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/DataReading.csv'

# Lists to store data and file details
merged_data = []
all_joint_positions_files = {}
all_labels_files = {}
deleted_files = []
subfolder_number = {}

In [3]:
# Traverse the main directory
for folder in tqdm(os.listdir(path_)):
    subfolder_path = os.path.join(path_, folder)
    
    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        # Count and store the number of items in this subfolder
        subfolder_number[folder] = len(os.listdir(subfolder_path))
        
        # Traverse each subdirectory in this subfolder
        for subfolder in os.listdir(subfolder_path):
            final_folder_path = os.path.join(subfolder_path, subfolder)
            
            # Check if it's a directory
            if os.path.isdir(final_folder_path):
                joint_positions_exists = False
                labels_exists = False
                joint_positions_file = None
                labels_file = None
                
                for file in os.listdir(final_folder_path):
                    full_file_path = os.path.join(final_folder_path, file)
                    
                    if file == 'Joint_Positions.csv':
                        joint_positions_file = full_file_path
                        joint_positions_exists = True
                    elif file == 'Labels.csv':
                        labels_file = full_file_path
                        labels_exists = True
                    else:
                        # Delete any file that is not 'Joint_Positions.csv' or 'Labels.csv'
                        os.remove(full_file_path)
                        deleted_files.append(full_file_path)
                        print(f"Deleted: {full_file_path}")
                
                # Print an error message if one of the files is missing
                if not joint_positions_exists:
                    print(f"Error: The path '{final_folder_path}' does not include the file 'Joint_Positions.csv'")
                if not labels_exists:
                    print(f"Error: The path '{final_folder_path}' does not include the file 'Labels.csv'")
                
                # Calculate rows and ratios if both files are present
                if joint_positions_exists and labels_exists:
                    try:
                        joint_df = pd.read_csv(joint_positions_file, header=None)
                        labels_df = pd.read_csv(labels_file, header=None)
                        joint_rows = len(joint_df)
                        labels_rows = len(labels_df)
                        
                        ratio = joint_rows / labels_rows if labels_rows > 0 else None
                        all_joint_positions_files[joint_positions_file] = joint_rows
                        all_labels_files[labels_file] = labels_rows
                        
                        print(f"File Pair: {joint_positions_file} and {labels_file}")
                        print(f"Joint_Positions.csv rows: {joint_rows}")
                        print(f"Labels.csv rows: {labels_rows}")
                        print(f"Ratio (Joint_Positions / Labels): {ratio:.2f}" if ratio is not None else "Error: Division by zero")
                        
                        # Process each row in labels file
                        for i in range(len(labels_df)):
                            output_row = labels_df.iloc[i].values
                            input_rows = joint_df.iloc[i * 25:(i + 1) * 25]
                            input_data = input_rows.values.flatten()
                            
                            # Concatenate input data (75 columns) with output data
                            merged_row = list(output_row) + list(input_data) + [folder, subfolder]
                            merged_data.append(merged_row)
                        
                    except Exception as e:
                        print(f"Error reading files in directory '{final_folder_path}': {e}")
        
        print('-----------------------------------\n')

  0%|          | 0/19 [00:00<?, ?it/s]

File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H01/Rch_Fwr_Bck_L/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H01/Rch_Fwr_Bck_L/Labels.csv
Joint_Positions.csv rows: 11525
Labels.csv rows: 461
Ratio (Joint_Positions / Labels): 25.00
File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H01/Rch_Fwr_Bck_LnFwr_L/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H01/Rch_Fwr_Bck_LnFwr_L/Labels.csv
Joint_Positions.csv rows: 12025
Labels.csv rows: 481
Ratio (Joint_Positions / Labels): 25.00
File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H01/Rch_Fwr_Bck_LnFwr_R/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H01/Rch_F

  5%|▌         | 1/19 [00:02<00:49,  2.73s/it]

File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H02/Rch_Fwr_Bck_L/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H02/Rch_Fwr_Bck_L/Labels.csv
Joint_Positions.csv rows: 12000
Labels.csv rows: 480
Ratio (Joint_Positions / Labels): 25.00
File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H02/Rch_Fwr_Bck_LnFwr_L/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H02/Rch_Fwr_Bck_LnFwr_L/Labels.csv
Joint_Positions.csv rows: 13175
Labels.csv rows: 527
Ratio (Joint_Positions / Labels): 25.00
File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H02/Rch_Fwr_Bck_LnFwr_R/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/H02/Rch_F

 11%|█         | 2/19 [00:05<00:43,  2.56s/it]

...

-----------------------------------

File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Fwr_Bck_L_1/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Fwr_Bck_L_1/Labels.csv
Joint_Positions.csv rows: 16900
Labels.csv rows: 676
Ratio (Joint_Positions / Labels): 25.00
File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Fwr_Bck_L_2/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Fwr_Bck_L_2/Labels.csv
Joint_Positions.csv rows: 17150
Labels.csv rows: 686
Ratio (Joint_Positions / Labels): 25.00
File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Fwr_Bck_R/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/C

 95%|█████████▍| 18/19 [00:36<00:01,  1.43s/it]

File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Sd2Sd_Bck_L/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P08/Rch_Sd2Sd_Bck_L/Labels.csv
Joint_Positions.csv rows: 20275
Labels.csv rows: 811
Ratio (Joint_Positions / Labels): 25.00
-----------------------------------

File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P09/Rch_Sd2Sd_Bck_L_1/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P09/Rch_Sd2Sd_Bck_L_1/Labels.csv
Joint_Positions.csv rows: 6000
Labels.csv rows: 240
Ratio (Joint_Positions / Labels): 25.00


100%|██████████| 19/19 [00:36<00:00,  1.92s/it]

File Pair: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P09/Rch_Sd2Sd_Bck_L_2/Joint_Positions.csv and /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/P09/Rch_Sd2Sd_Bck_L_2/Labels.csv
Joint_Positions.csv rows: 5225
Labels.csv rows: 209
Ratio (Joint_Positions / Labels): 25.00
-----------------------------------



In [4]:
# Print summary
print(f"\nSummary:")
print(f"Total Labels.csv files retained: {len(all_labels_files)}")
print(f"Total Joint_Positions.csv files retained: {len(all_joint_positions_files)}")
print(f"Total files deleted: {len(deleted_files)}")


Summary:
Total Labels.csv files retained: 149
Total Joint_Positions.csv files retained: 149
Total files deleted: 0


In [5]:
# Create DataFrame and save to CSV
try:
    # Define headers
    header = ['OUTPUT']  # First column header

    # Add Sample headers
    for i in range(1, 26):  # For 25 samples
        header.extend([f'Sample_{i}_X', f'Sample_{i}_Y', f'Sample_{i}_Z'])
    
    # Add new columns
    header.extend(['Main_Folder', 'Sub_Folder'])
    
    # Create DataFrame and assign header
    merged_df = pd.DataFrame(merged_data)
    merged_df.columns = header
    
    # Save the DataFrame with header to a new CSV file
    merged_df.to_csv(headered_output_path, index=False)
    print(f"Merged file with header saved to: {headered_output_path}")
except Exception as e:
    print(f"Error saving merged file: {e}")

Merged file with header saved to: /media/ahmed-ferganey/AI4/01-Learning_AI/MyGitHub/Machine_Learning_Projects/CSV_Files/data_new/DataReading.csv


In [6]:
DataReadingFrame = pd.read_csv(headered_output_path)
DataReadingFrame

,OUTPUT,Sample_1_X,Sample_1_Y,Sample_1_Z,Sample_2_X,Sample_2_Y,Sample_2_Z,Sample_3_X,Sample_3_Y,Sample_3_Z,Sample_4_X,Sample_4_Y,Sample_4_Z,Sample_5_X,Sample_5_Y,Sample_5_Z,Sample_6_X,Sample_6_Y,Sample_6_Z,Sample_7_X,Sample_7_Y,Sample_7_Z,Sample_8_X,Sample_8_Y,Sample_8_Z,Sample_9_X,Sample_9_Y,Sample_9_Z,Sample_10_X,Sample_10_Y,Sample_10_Z,Sample_11_X,Sample_11_Y,Sample_11_Z,Sample_12_X,Sample_12_Y,Sample_12_Z,Sample_13_X,Sample_13_Y,Sample_13_Z,Sample_14_X,Sample_14_Y,Sample_14_Z,Sample_15_X,Sample_15_Y,Sample_15_Z,Sample_16_X,Sample_16_Y,Sample_16_Z,Sample_17_X,Sample_17_Y,Sample_17_Z,Sample_18_X,Sample_18_Y,Sample_18_Z,Sample_19_X,Sample_19_Y,Sample_19_Z,Sample_20_X,Sample_20_Y,Sample_20_Z,Sample_21_X,Sample_21_Y,Sample_21_Z,Sample_22_X,Sample_22_Y,Sample_22_Z,Sample_23_X,Sample_23_Y,Sample_23_Z,Sample_24_X,Sample_24_Y,Sample_24_Z,Sample_25_X,Sample_25_Y,Sample_25_Z,Main_Folder,Sub_Folder
0,1,-0.126229,-1.077193,-0.372164,-0.059609,-0.449359,-0.142584,0.022335,0.157045,0.047985,0.020157,0.574201,0.051572,-0.527653,-0.039080,-0.005213,-0.730828,-0.524220,-0.225948,-0.675773,-0.530603,-0.860346,-0.592214,-0.525006,-1.022693,0.461188,-0.188052,-0.004890,0.671840,-0.628131,-0.202664,0.789949,-0.773413,-0.641858,0.811696,-0.809308,-0.752797,-0.292051,-1.008281,-0.455261,0.111003,-0.670195,-1.174915,0.016962,-1.678548,-1.552268,0.241415,-1.315421,-1.664223,0.141221,-1.078490,-0.417798,0.775491,-0.719739,-1.061692,0.670538,-1.726800,-1.436622,0.865867,-1.343595,-1.505179,-0.003054,-0.003911,0.003185,-0.487667,-0.512424,-1.170564,-0.474437,-0.414736,-1.108108,0.841255,-0.836067,-0.985780,0.704939,-0.720179,-0.823998,H01,Rch_Fwr_Bck_L
1,1,-0.122184,-1.064600,-0.370825,-0.057122,-0.446842,-0.145189,0.023782,0.157384,0.045812,0.022817,0.574343,0.047328,-0.526403,-0.038921,-0.007104,-0.729617,-0.556266,-0.241112,-0.674120,-0.532842,-0.877907,-0.568733,-0.519463,-1.093277,0.462477,-0.187635,-0.006938,0.668347,-0.628704,-0.204829,0.791148,-0.773826,-0.644721,0.814159,-0.809213,-0.754541,-0.288798,-0.997050,-0.453150,0.110981,-0.669748,-1.175493,0.014848,-1.678228,-1.550837,0.237942,-1.318614,-1.662326,0.144168,-1.066164,-0.415090,0.776837,-0.719129,-1.063724,0.671560,-1.726398,-1.438141,0.854623,-1.341478,-1.503475,-0.001165,-0.001975,0.001207,-0.622477,-0.593360,-1.308104,-0.467350,-0.434511,-1.147998,0.842592,-0.845569,-0.977329,0.727899,-0.710592,-0.822619,H01,Rch_Fwr_Bck_L
2,1,-0.116582,-1.044229,-0.367488,-0.054663,-0.441613,-0.146506,0.024943,0.157655,0.043976,0.025081,0.574477,0.043777,-0.525354,-0.038701,-0.008662,-0.705656,-0.578459,-0.207481,-0.692328,-0.566002,-0.708448,-0.605477,-0.527531,-1.007742,0.463547,-0.187272,-0.008675,0.666312,-0.626723,-0.205890,0.791796,-0.773707,-0.645759,0.813976,-0.809073,-0.751123,-0.283966,-0.977481,-0.448594,0.114750,-0.669194,-1.179361,0.017370,-1.677729,-1.553436,0.236494,-1.323303,-1.662935,0.148945,-1.046690,-0.411347,0.777934,-0.718586,-1.065326,0.672399,-1.725980,-1.439312,0.850888,-1.346205,-1.502556,0.000335,0.000168,-0.000414,-0.506695,-0.525854,-1.170112,-0.487626,-0.434253,-1.084617,0.834789,-0.847251,-0.966044,0.729321,-0.711991,-0.825931,H01,Rch_Fwr_Bck_L
3,1,-0.113412,-1.022981,-0.362565,-0.052826,-0.433942,-0.146035,0.025874,0.157815,0.042454,0.027027,0.574299,0.040643,-0.524714,-0.038902,-0.009903,-0.715591,-0.581668,-0.224163,-0.636173,-0.537277,-0.862396,-0.560913,-0.513314,-1.083591,0.464280,-0.186857,-0.010152,0.710866,-0.627741,-0.193704,0.791766,-0.773187,-0.646145,0.813366,-0.809360,-0.754943,-0.281897,-0.957549,-0.441853,0.115744,-0.668439,-1.180320,0.018091,-1.676607,-1.553820,0.235405,-1.324502,-1.662334,0.151823,-1.025675,-0.406761,0.778441,-0.717797,-1.066285,0.672802,-1.724847,-1.439765,0.857071,-1.350854,-1.504357,0.001522,0.002572,-0.001337,-0.616487,-0.582546,-1.288381,-0.469262,-0.440891,-1.151198,0.838387,-0.839690,-0.975070,0.724226,-0.713290,-0.826963,H01,Rch_Fwr_Bck_L
4,1,-0.112263,-1.023139,-0.365561,-0.051589,-0.433444,-0.148052,0.026449,0.157805,0.041304

In [7]:
DataReadingFrame['OUTPUT'].unique()

array([1, 4, 2, 3])

In [8]:
DataReadingFrame['OUTPUT'].value_counts()

OUTPUT
1    51800
3     9053
2     5278
4     4714
Name: count, dtype: int64

In [9]:
DataReadingFrame['Main_Folder'].value_counts()

Main_Folder
H09    7556
H01    5376
H07    5183
H02    4944
H06    4944
H05    4788
H04    4577
H08    4315
H03    4125
H10    4102
P06    3975
P08    2864
P01    2840
P04    2518
P05    2296
P07    2261
P03    1976
P02    1756
P09     449
Name: count, dtype: int64

In [10]:
DataReadingFrame['Sub_Folder'].value_counts()

Sub_Folder
Rch_Sd2Sd_Bck_L          8381
Rch_Fwr_Bck_L            7019
Rch_Fwr_Bck_R            6720
Rch_Sd2Sd_Bck_R          6171
Rch_Sd2Sd_Bck_TrRot_R    4676
Rch_Sd2Sd_Bck_TrRot_L    4550
Rch_Fwr_Bck_TrRot_L      4473
Rch_Fwr_Bck_LnFwr_L      4473
Rch_Fwr_Bck_ShElev_L     4412
Rch_Fwr_Bck_LnFwr_R      4383
Rch_Fwr_Bck_TrRot_R      4001
Rch_Fwr_Bck_ShElev_R     3893
Rch_Sd2Sd_Bck_L_2        1573
Rch_Sd2Sd_Bck_L_1        1497
Rch_Sd2Sd_Bck_R_2        1252
Rch_Fwr_Bck_R_2           920
Rch_Fwr_Bck_L_2           686
Rch_Fwr_Bck_L_1           676
Rch_Sd2Sd_Bck_R_1         588
Rch_Fwr_Bck_R_1           501
Name: count, dtype: int64

### 4. data analysis
___

In [12]:
AnalyticsData = DataReadingFrame

In [13]:
AnalyticsData

,OUTPUT,Sample_1_X,Sample_1_Y,Sample_1_Z,Sample_2_X,Sample_2_Y,Sample_2_Z,Sample_3_X,Sample_3_Y,Sample_3_Z,Sample_4_X,Sample_4_Y,Sample_4_Z,Sample_5_X,Sample_5_Y,Sample_5_Z,Sample_6_X,Sample_6_Y,Sample_6_Z,Sample_7_X,Sample_7_Y,Sample_7_Z,Sample_8_X,Sample_8_Y,Sample_8_Z,Sample_9_X,Sample_9_Y,Sample_9_Z,Sample_10_X,Sample_10_Y,Sample_10_Z,Sample_11_X,Sample_11_Y,Sample_11_Z,Sample_12_X,Sample_12_Y,Sample_12_Z,Sample_13_X,Sample_13_Y,Sample_13_Z,Sample_14_X,Sample_14_Y,Sample_14_Z,Sample_15_X,Sample_15_Y,Sample_15_Z,Sample_16_X,Sample_16_Y,Sample_16_Z,Sample_17_X,Sample_17_Y,Sample_17_Z,Sample_18_X,Sample_18_Y,Sample_18_Z,Sample_19_X,Sample_19_Y,Sample_19_Z,Sample_20_X,Sample_20_Y,Sample_20_Z,Sample_21_X,Sample_21_Y,Sample_21_Z,Sample_22_X,Sample_22_Y,Sample_22_Z,Sample_23_X,Sample_23_Y,Sample_23_Z,Sample_24_X,Sample_24_Y,Sample_24_Z,Sample_25_X,Sample_25_Y,Sample_25_Z,Main_Folder,Sub_Folder
0,1,-0.126229,-1.077193,-0.372164,-0.059609,-0.449359,-0.142584,0.022335,0.157045,0.047985,0.020157,0.574201,0.051572,-0.527653,-0.039080,-0.005213,-0.730828,-0.524220,-0.225948,-0.675773,-0.530603,-0.860346,-0.592214,-0.525006,-1.022693,0.461188,-0.188052,-0.004890,0.671840,-0.628131,-0.202664,0.789949,-0.773413,-0.641858,0.811696,-0.809308,-0.752797,-0.292051,-1.008281,-0.455261,0.111003,-0.670195,-1.174915,0.016962,-1.678548,-1.552268,0.241415,-1.315421,-1.664223,0.141221,-1.078490,-0.417798,0.775491,-0.719739,-1.061692,0.670538,-1.726800,-1.436622,0.865867,-1.343595,-1.505179,-0.003054,-0.003911,0.003185,-0.487667,-0.512424,-1.170564,-0.474437,-0.414736,-1.108108,0.841255,-0.836067,-0.985780,0.704939,-0.720179,-0.823998,H01,Rch_Fwr_Bck_L
1,1,-0.122184,-1.064600,-0.370825,-0.057122,-0.446842,-0.145189,0.023782,0.157384,0.045812,0.022817,0.574343,0.047328,-0.526403,-0.038921,-0.007104,-0.729617,-0.556266,-0.241112,-0.674120,-0.532842,-0.877907,-0.568733,-0.519463,-1.093277,0.462477,-0.187635,-0.006938,0.668347,-0.628704,-0.204829,0.791148,-0.773826,-0.644721,0.814159,-0.809213,-0.754541,-0.288798,-0.997050,-0.453150,0.110981,-0.669748,-1.175493,0.014848,-1.678228,-1.550837,0.237942,-1.318614,-1.662326,0.144168,-1.066164,-0.415090,0.776837,-0.719129,-1.063724,0.671560,-1.726398,-1.438141,0.854623,-1.341478,-1.503475,-0.001165,-0.001975,0.001207,-0.622477,-0.593360,-1.308104,-0.467350,-0.434511,-1.147998,0.842592,-0.845569,-0.977329,0.727899,-0.710592,-0.822619,H01,Rch_Fwr_Bck_L
2,1,-0.116582,-1.044229,-0.367488,-0.054663,-0.441613,-0.146506,0.024943,0.157655,0.043976,0.025081,0.574477,0.043777,-0.525354,-0.038701,-0.008662,-0.705656,-0.578459,-0.207481,-0.692328,-0.566002,-0.708448,-0.605477,-0.527531,-1.007742,0.463547,-0.187272,-0.008675,0.666312,-0.626723,-0.205890,0.791796,-0.773707,-0.645759,0.813976,-0.809073,-0.751123,-0.283966,-0.977481,-0.448594,0.114750,-0.669194,-1.179361,0.017370,-1.677729,-1.553436,0.236494,-1.323303,-1.662935,0.148945,-1.046690,-0.411347,0.777934,-0.718586,-1.065326,0.672399,-1.725980,-1.439312,0.850888,-1.346205,-1.502556,0.000335,0.000168,-0.000414,-0.506695,-0.525854,-1.170112,-0.487626,-0.434253,-1.084617,0.834789,-0.847251,-0.966044,0.729321,-0.711991,-0.825931,H01,Rch_Fwr_Bck_L
3,1,-0.113412,-1.022981,-0.362565,-0.052826,-0.433942,-0.146035,0.025874,0.157815,0.042454,0.027027,0.574299,0.040643,-0.524714,-0.038902,-0.009903,-0.715591,-0.581668,-0.224163,-0.636173,-0.537277,-0.862396,-0.560913,-0.513314,-1.083591,0.464280,-0.186857,-0.010152,0.710866,-0.627741,-0.193704,0.791766,-0.773187,-0.646145,0.813366,-0.809360,-0.754943,-0.281897,-0.957549,-0.441853,0.115744,-0.668439,-1.180320,0.018091,-1.676607,-1.553820,0.235405,-1.324502,-1.662334,0.151823,-1.025675,-0.406761,0.778441,-0.717797,-1.066285,0.672802,-1.724847,-1.439765,0.857071,-1.350854,-1.504357,0.001522,0.002572,-0.001337,-0.616487,-0.582546,-1.288381,-0.469262,-0.440891,-1.151198,0.838387,-0.839690,-0.975070,0.724226,-0.713290,-0.826963,H01,Rch_Fwr_Bck_L
4,1,-0.112263,-1.023139,-0.365561,-0.051589,-0.433444,-0.148052,0.026449,0.157805,0.041304

### 5. data cleaning
___

##### 5.1 finding nulls

##### 5.2 outliers

##### 5.3 feature extraction

##### 5.4 feature selection

### 6. visualization
___

### 7. building the model
___

### 8. evaluation the model
___